<a href="https://colab.research.google.com/github/VinhDo0608/Do_An/blob/main/%C4%90%E1%BB%93_%C3%A1n.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [14]:
from keras.applications import vgg16
from keras.models import Sequential
from keras.layers import Flatten, Dense, BatchNormalization, Dropout
from google.colab import files
import ipywidgets as widgets
from IPython.display import display, clear_output
from PIL import Image
import tensorflow as tf
import numpy as np
import io
from google.colab.patches import cv2_imshow
import cv2
import matplotlib.pyplot as plt
import os
from keras.preprocessing import image
import keras
from keras.callbacks import ModelCheckpoint
import random
from keras.callbacks import ModelCheckpoint
import shutil
from keras.preprocessing.image import ImageDataGenerator

In [10]:
def build_custom_model(input_shape=(128, 128, 3), num_classes=7):
    # Tạo mô hình VGG16 đã được đào tạo trên ImageNet
    vgg = vgg16.VGG16(weights='imagenet',
                      include_top=False,
                      input_shape=input_shape)

    # Đóng băng trọng số của các tầng VGG16 (tầng đã được đào tạo)
    for layer in vgg.layers:
        layer.trainable = False

    # Xây dựng mô hình dạng tuần tự
    model = Sequential()

    # Thêm mô hình VGG16 đã được đào tạo vào mô hình tuần tự
    model.add(vgg)

    # Làm phẳng đầu ra của VGG16 trước khi kết nối vào các tầng fully connected (FC)
    model.add(Flatten())

    # Các tầng FC để thực hiện phân loại
    model.add(Dense(256))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))

    model.add(Dense(256))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))

    model.add(Dense(128))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))

    # Tầng đầu ra với số lượng đơn vị tương ứng với số lớp và kích hoạt softmax
    model.add(Dense(num_classes, activation='softmax'))

    # Hiển thị tóm tắt của mô hình
    model.summary()

    return model

model = build_custom_model()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 4, 4, 512)         14714688  
                                                                 
 flatten_1 (Flatten)         (None, 8192)              0         
                                                                 
 dense_4 (Dense)             (None, 256)               2097408   
                                                                 
 batch_normalization_3 (Bat  (None, 256)               1024      
 chNormalization)                                                
                                                                 
 dropout_3 (Dropout)         (None, 256)               0         
                                                                 
 dense_5 (Dense)             (None, 256)               65792     
                                                      

In [ ]:
for dirname, _, filenames in os.walk('/content/drive/MyDrive/Colab Notebooks/DATA/Cucumber_diesease'):
    for filename in filenames:
        # In ra đường dẫn đầy đủ của từng tệp
        print(os.path.join(dirname, filename))

In [7]:
# Đường dẫn tới thư mục chứa dữ liệu ảnh
data_dir = '/content/drive/MyDrive/Colab Notebooks/DATA/Cucumber_diesease'

# Số lượng ảnh trong mỗi lô (batch) sử dụng trong quá trình đào tạo
batch_size = 64

# Chiều cao của ảnh (ảnh sẽ được chuyển về kích thước này)
img_height = 128

# Chiều rộng của ảnh (ảnh sẽ được chuyển về kích thước này)
img_width = 128


In [8]:
train_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size)

Found 15982 files belonging to 7 classes.
Using 12786 files for training.


In [ ]:
val_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size)

In [ ]:
class_names = train_ds.class_names
print("Tên các loại bệnh")
print(class_names)

In [ ]:
def display_images_from_folders(main_folder_path, num_rows=4, num_cols=2):
    subfolders = [f.path for f in os.scandir(main_folder_path) if f.is_dir()]
    fig, axes = plt.subplots(num_rows, num_cols, figsize=(12, 18))

    for i, subfolder in enumerate(subfolders):
        image_files = [f.path for f in os.scandir(subfolder) if f.is_file() and f.name.endswith((".jpg", ".jpeg", ".png"))]

        if image_files:
            first_image_path = image_files[0]
            image = cv2.imread(first_image_path)
            image_resized = cv2.resize(image, (128, 128))

            axes[i // num_cols, i % num_cols].imshow(cv2.cvtColor(image_resized, cv2.COLOR_BGR2RGB))
            axes[i // num_cols, i % num_cols].set_title(os.path.basename(subfolder))
            axes[i // num_cols, i % num_cols].axis('off')

    plt.show()

# Sử dụng hàm để hiển thị các ảnh từ thư mục chính
main_folder_path = "/content/drive/MyDrive/Colab Notebooks/DATA/Cucumber_diesease"
display_images_from_folders(main_folder_path)

In [ ]:
checkpoint = ModelCheckpoint("/content/drive/MyDrive/Training/model_weights_new_t4.h5",
                             monitor="val_loss",
                             mode="min",
                             save_best_only = True,
                             verbose=1)

In [ ]:
model.compile(optimizer='adam',
              loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
epochs = 10
history = model.fit(
    train_ds,                # Tập dữ liệu đào tạo
    steps_per_epoch=20,       # Số lần lặp lại mỗi epoch
    verbose=1,               # Mức độ in thông tin
    epochs=epochs,           # Số lượng epoch (vòng lặp qua toàn bộ tập dữ liệu đào tạo)
    callbacks=checkpoint,    # Danh sách các callback (ở đây là "checkpoint")
    validation_data=val_ds   # Tập dữ liệu validation
)

In [ ]:
def plot_training_history(history):
    # Lấy giá trị độ chính xác trên tập huấn luyện và tập kiểm tra từ lịch sử
    training_accuracy = history.history['accuracy']
    validation_accuracy = history.history['val_accuracy']

    # Lấy số epochs
    epochs = range(1, len(training_accuracy) + 1)

    # Vẽ đồ thị
    plt.figure(figsize=(10, 6))
    plt.plot(epochs, training_accuracy, label='Training Accuracy')
    plt.plot(epochs, validation_accuracy, label='Validation Accuracy')
    plt.title('Training and Validation Accuracy Over Epochs')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.show()

# Giả sử "history" là biến chứa lịch sử huấn luyện từ model.fit
# Chẳng hạn: history = model.fit(train_data, epochs=10, validation_data=validation_data)
plot_training_history(history)

In [ ]:
def plot_training_history(history):
    # Lấy giá trị độ chính xác trên tập huấn luyện và tập kiểm tra từ lịch sử
    training_accuracy = history.history['loss']
    validation_accuracy = history.history['val_loss']

    # Lấy số epochs
    epochs = range(1, len(training_accuracy) + 1)

    # Vẽ đồ thị
    plt.figure(figsize=(10, 6))
    plt.plot(epochs, training_accuracy, label='loss')
    plt.plot(epochs, validation_accuracy, label='val_loss')
    plt.title('Loss and Val loss Over Epochs')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()

# Giả sử "history" là biến chứa lịch sử huấn luyện từ model.fit
# Chẳng hạn: history = model.fit(train_data, epochs=10, validation_data=validation_data)
plot_training_history(history)

In [ ]:
def create_test_set(data_dir, test_dir, test_ratio=0.01):
    # Lấy danh sách tên các thư mục trong data_dir
    folders = os.listdir(data_dir)

    # Tạo thư mục test set nếu chưa tồn tại
    if not os.path.exists(test_dir):
        os.makedirs(test_dir)

    # Lặp qua từng thư mục
    for folder in folders:
        folder_path = os.path.join(data_dir, folder)

        # Kiểm tra xem có phải là thư mục không
        if os.path.isdir(folder_path):
            # Lấy danh sách các tệp trong thư mục
            files = os.listdir(folder_path)

            # Số lượng tệp để chuyển sang test set
            num_files_to_move = int(len(files) * test_ratio)

            # Lựa chọn ngẫu nhiên các tệp
            files_to_move = random.sample(files, num_files_to_move)

            # Tạo đường dẫn đến thư mục test set cho thư mục hiện tại
            test_folder_path = os.path.join(test_dir, folder)

            # Tạo thư mục test set cho thư mục hiện tại nếu chưa tồn tại
            if not os.path.exists(test_folder_path):
                os.makedirs(test_folder_path)

            # Di chuyển các tệp vào thư mục test set
            for file in files_to_move:
                file_path = os.path.join(folder_path, file)
                dest_path = os.path.join(test_folder_path, file)
                shutil.move(file_path, dest_path)
    print("Test set created successfully.")

# Sử dụng hàm để tạo tập test
data_dir = "/content/drive/MyDrive/Colab Notebooks/DATA/Cucumber_diesease"
test_dir = "/content/drive/MyDrive/Colab Notebooks/DATA/test_set"
create_test_set(data_dir, test_dir, test_ratio=0.01)

In [ ]:
def count_images_in_folders(main_folder):
    # Khởi tạo biến để đếm tổng số lượng ảnh
    total_images = 0

    # Tên các thư mục con (assumed to be class labels)
    class_folders = os.listdir(main_folder)

    # Duyệt qua từng thư mục con và tính tổng số lượng ảnh
    for class_folder in class_folders:
        class_path = os.path.join(main_folder, class_folder)

        # Đảm bảo thư mục là một thư mục và không phải là tệp tin
        if os.path.isdir(class_path):
            # List tất cả các tệp tin hình ảnh trong thư mục con
            image_files = [f for f in os.listdir(class_path) if f.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp', '.tiff'))]

            # Tính tổng số lượng ảnh trong thư mục con và cộng vào tổng chung
            total_images += len(image_files)

    # Đưa ra tổng số lượng ảnh
    print(f"Tổng số lượng ảnh: {total_images}")

# Sử dụng hàm để in ra tổng số lượng ảnh
main_folder = "/content/drive/MyDrive/Colab Notebooks/DATA/test_set"
count_images_in_folders(main_folder)

In [ ]:
def evaluate_model(model_path, test_folder):
    # Load the trained model
    model = keras.models.load_model(model_path)

    # Tên các thư mục con (giả sử là nhãn lớp)
    class_folders = os.listdir(test_folder)

    # Khởi tạo biến đếm số dự đoán đúng
    correct_predictions = 0

    # Khởi tạo biến để đếm tổng số lượng ảnh
    total_images = 0

    # Duyệt qua từng thư mục lớp
    for class_folder in class_folders:
        class_path = os.path.join(test_folder, class_folder)

        # Đảm bảo thư mục là một thư mục và không phải là tệp tin
        if os.path.isdir(class_path):
            # List tất cả các tệp tin hình ảnh trong thư mục lớp
            image_files = [f for f in os.listdir(class_path) if f.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp', '.tiff'))]

            # Tính tổng số lượng ảnh trong thư mục lớp và cộng vào tổng chung
            total_images += len(image_files)

            # Lặp lại từng hình ảnh trong thư mục lớp
            for image_file in image_files:
                img_path = os.path.join(class_path, image_file)
                # Tải và xử lý trước hình ảnh để dự đoán
                img = image.load_img(img_path, target_size=(128, 128))
                img_array = image.img_to_array(img)
                img_array = np.expand_dims(img_array, axis=0)

                # Dự đoán nhãn cho hình ảnh
                y_pred = model.predict(img_array)
                predicted_class = np.argmax(y_pred)
                accuracy = y_pred[0][predicted_class]

                # Nhận tên lớp (thay thế bằng tên lớp thực tế của bạn)
                class_names = ['Anthracnose', 'Bacterial Wilt', 'Belly Rot', 'Downy Mildew', 'Fresh Leaf', 'Good', 'Gummy Stem Blight']

                # Kiểm tra xem dự đoán có đúng không
                true_class = class_folders.index(class_folder)
                if predicted_class == true_class:
                    correct_predictions += 1

                # Hiển thị hình ảnh và thông tin dự đoán
                plt.figure(figsize=(6, 6))
                plt.imshow(img)
                plt.title(f"True label: {class_names[true_class]}\nPredicted: {class_names[predicted_class]}\nAccuracy: {accuracy:.2%}\nFile: {image_file}")
                plt.axis('off')
                plt.show()

    # Tính tỷ lệ dự đoán đúng trên tập test
    total_accuracy = (correct_predictions / total_images) * 100
    print(f"Tỉ lệ chính xác của tập test: {total_accuracy:.2f}%")

# Sử dụng hàm để đánh giá mô hình trên tập test
model_path = "/content/drive/MyDrive/Training/model_weights_new_t4.h5"
test_folder = "/content/drive/MyDrive/Colab Notebooks/DATA/test_set"
evaluate_model(model_path, test_folder)

In [ ]:
# Load model
model = keras.models.load_model("/content/drive/MyDrive/Training/model_weights_new_t4.h5")  # Thay thế đường dẫn đến mô hình của bạn

# Load and preprocess the image for prediction
img_path = '/content/drive/MyDrive/Colab Notebooks/DATA/Cucumber_diesease/Belly Rot/Belly Rot (3)_270.jpg'
img = image.load_img(img_path, target_size=(128, 128))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)  # Reshape to (1, 128, 128, 3) for a single image

y_pred = model.predict(img_array)
predicted_class = np.argmax(y_pred)
accuracy = y_pred[0][predicted_class]

class_names = ['Anthracnose', 'Bacterial Wilt', 'Belly Rot', 'Downy Mildew', 'Fresh Leaf', 'Gummy Stem Blight','Good']
# Hiển thị độ chính xác
print(f"Predicted class: {class_names[predicted_class]}")
print(f"Accuracy: {accuracy:.2%}")

In [ ]:
# Đường dẫn đến thư mục chứa dữ liệu huấn luyện
train_dir = "/content/drive/MyDrive/Colab Notebooks/DATA/Cucumber_diesease"

# Đường dẫn đến thư mục chứa dữ liệu kiểm thử
test_dir = "/content/drive/MyDrive/Colab Notebooks/DATA/test_set"

# Kích thước ảnh mà mô hình mong đợi
target_size = (128, 128)

# Số lượng ảnh được đọc mỗi lần từ thư mục
batch_size = 32

# Tạo generator cho dữ liệu huấn luyện
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=target_size,
    batch_size=batch_size,
    class_mode='categorical',  # Chọn 'binary' nếu chỉ có hai lớp
    shuffle=True
)

# Tạo generator cho dữ liệu kiểm thử
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=target_size,
    batch_size=batch_size,
    class_mode='categorical',  # Chọn 'binary' nếu chỉ có hai lớp
    shuffle=False
)

In [ ]:
# Load pre-trained model (make sure you have a pre-trained model)
model = tf.keras.models.load_model('/content/drive/MyDrive/Training/model_weights_new_t4.h5')

class PlantDiseaseDetectorApp:
    def __init__(self):
        self.label = widgets.Label("Chọn ảnh cây cần kiểm tra:")
        display(self.label)

        self.btn_upload = widgets.FileUpload(description="Upload Image", style={'button_color': 'lightblue'})
        self.btn_upload.observe(self.on_upload_change, names=['value'])
        display(self.btn_upload)

        self.image_output = widgets.Output()
        display(self.image_output)

        self.btn_clear = widgets.Button(description="Xóa kết quả", style={'button_color': 'salmon'})
        self.btn_clear.on_click(self.clear_result)
        display(self.btn_clear)

        self.result_label = widgets.Label("")
        display(self.result_label)

    def on_upload_change(self, change):
        for name, file_info in self.btn_upload.value.items():
            content = file_info['content']
            self.display_image(content)
            self.predict_disease(content, name)

    def display_image(self, content):
        image = Image.open(io.BytesIO(content))
        image = image.resize((400, 400))
        display(image)

    def predict_disease(self, content, file_name):
        image_path = os.path.join("/content", file_name)
        with open(image_path, "wb") as f:
            f.write(content)

        # Load and preprocess the image for prediction
        img = Image.open(image_path)
        img = img.resize((128, 128))
        img_array = tf.keras.preprocessing.image.img_to_array(img)
        img_array = tf.expand_dims(img_array, 0)


        y_pred = model.predict(img_array)
        predicted_class = np.argmax(y_pred)
        accuracy = y_pred[0][predicted_class]

        class_names = train_ds.class_names

        result = f"Loại bệnh: {class_names[predicted_class]} (Độ chắc chắn: {100 * accuracy:.2f}%)"
        self.result_label.value = result

        # Remove the temporary image file
        os.remove(image_path)

    def clear_result(self, _):
        clear_output(wait=True)
        self.btn_upload.value.clear()
        self.result_label.value = ""
        display(self.label, self.btn_upload, self.image_output, self.result_label, self.btn_clear)

# Create an app and display it on Colab
app = PlantDiseaseDetectorApp()
